# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [4]:
# Do all imports and installs here
import pandas as pd

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [3]:
df_spark =spark.read.load('./sas_data')

In [5]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
## Describe I94 Immigration Data

In [11]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [12]:
df_i94.head()

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode i94addr  \
0    6.0  2016.0     4.0   692.0   692.0     XXX  20573.0      NaN     NaN   
1    7.0  2016.0     4.0   254.0   276.0     ATL  20551.0      1.0      AL   
2   15.0  2016.0     4.0   101.0   101.0     WAS  20545.0      1.0      MI   
3   16.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   
4   17.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   

   depdate   ...     entdepu  matflag  biryear   dtaddto gender insnum  \
0      NaN   ...           U      NaN   1979.0  10282016    NaN    NaN   
1      NaN   ...           Y      NaN   1991.0       D/S      M    NaN   
2  20691.0   ...         NaN        M   1961.0  09302016      M    NaN   
3  20567.0   ...         NaN        M   1988.0  09302016    NaN    NaN   
4  20567.0   ...         NaN        M   2012.0  09302016    NaN    NaN   

  airline        admnum  fltno visatype  
0     NaN  1.897628e+09    NaN       B2  
1     NaN  3.736796e+09  00296       F1  
2      OS  6.666432e+08     93       B2  
3      AA  9.246846e+10  00199       B2  
4      AA  9.246846e+10  00199       B2  

[5 rows x 28 columns]

In [13]:
df_i94.describe()

cicid      i94yr     i94mon        i94cit        i94res  \
count  3.096313e+06  3096313.0  3096313.0  3.096313e+06  3.096313e+06   
mean   3.078652e+06     2016.0        4.0  3.049069e+02  3.032838e+02   
std    1.763278e+06        0.0        0.0  2.100269e+02  2.085832e+02   
min    6.000000e+00     2016.0        4.0  1.010000e+02  1.010000e+02   
25%    1.577790e+06     2016.0        4.0  1.350000e+02  1.310000e+02   
50%    3.103507e+06     2016.0        4.0  2.130000e+02  2.130000e+02   
75%    4.654341e+06     2016.0        4.0  5.120000e+02  5.040000e+02   
max    6.102785e+06     2016.0        4.0  9.990000e+02  7.600000e+02   

            arrdate       i94mode       depdate        i94bir       i94visa  \
count  3.096313e+06  3.096074e+06  2.953856e+06  3.095511e+06  3.096313e+06   
mean   2.055985e+04  1.073690e+00  2.057395e+04  4.176761e+01  1.845393e+00   
std    8.777339e+00  5.158963e-01  2.935697e+01  1.742026e+01  3.983910e-01   
min    2.054500e+04  1.000000e+00  1.517600e+04 -3.000000e+00  1.000000e+00   
25%    2.055200e+04  1.000000e+00  2.056100e+04  3.000000e+01  2.000000e+00   
50%    2.056000e+04  1.000000e+00  2.057000e+04  4.100000e+01  2.000000e+00   
75%    2.056700e+04  1.000000e+00  2.057900e+04  5.400000e+01  2.000000e+00   
max    2.057400e+04  9.000000e+00  4.542700e+04  1.140000e+02  3.000000e+00   

           count       biryear        admnum  
count  3096313.0  3.095511e+06  3.096313e+06  
mean         1.0  1.974232e+03  7.082885e+10  
std          0.0  1.742026e+01  2.215442e+10  
min          1.0  1.902000e+03  0.000000e+00  
25%          1.0  1.962000e+03  5.603523e+10  
50%          1.0  1.975000e+03  5.936094e+10  
75%          1.0  1.986000e+03  9.350987e+10  
max          1.0  2.019000e+03  9.991557e+10

In [15]:
df_i94.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [19]:
df_i94.count()

cicid       3096313
i94yr       3096313
i94mon      3096313
i94cit      3096313
i94res      3096313
i94port     3096313
arrdate     3096313
i94mode     3096074
i94addr     2943941
depdate     2953856
i94bir      3095511
i94visa     3096313
count       3096313
dtadfile    3096312
visapost    1215063
occup          8126
entdepa     3096075
entdepd     2957884
entdepu         392
matflag     2957884
biryear     3095511
dtaddto     3095836
gender      2682044
insnum       113708
airline     3012686
admnum      3096313
fltno       3076764
visatype    3096313
dtype: int64

In [16]:
## Describe World Temperature Data

In [4]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_tem = pd.read_csv(fname)
df_tem.iloc[:10000, :].to_csv('GlobalLandTemperaturesByCity.csv')

In [18]:
df_tem.head()

dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E

In [10]:
df_tem.count()

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [8]:
#write to parquet
# df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

In [9]:
df_spark.describe()

DataFrame[summary: string, cicid: string, i94yr: string, i94mon: string, i94cit: string, i94res: string, i94port: string, arrdate: string, i94mode: string, i94addr: string, depdate: string, i94bir: string, i94visa: string, count: string, dtadfile: string, visapost: string, occup: string, entdepa: string, entdepd: string, entdepu: string, matflag: string, biryear: string, dtaddto: string, gender: string, insnum: string, airline: string, admnum: string, fltno: string, visatype: string]

In [10]:
df_spark.summary().show()

+-------+------------------+-------+-------+------------------+------------------+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+-------+--------------------+--------+-----------------+-------+-------+-------+-------+------------------+------------------+-------+-----------------+------------------+--------------------+------------------+--------+
|summary|             cicid|  i94yr| i94mon|            i94cit|            i94res|i94port|          arrdate|           i94mode|           i94addr|          depdate|            i94bir|           i94visa|  count|            dtadfile|visapost|            occup|entdepa|entdepd|entdepu|matflag|           biryear|           dtaddto| gender|           insnum|           airline|              admnum|             fltno|visatype|
+-------+------------------+-------+-------+------------------+------------------+-------+-----------------+------------------+------------------+--------

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [11]:
# Performing cleaning tasks here



1


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.